In [ ]:
from dotenv import load_dotenv
import os
from main import read_website_content
from langchain_groq import ChatGroq
from IPython.display import display, Markdown

In [ ]:
load_dotenv()

In [ ]:
groq_llm = ChatGroq(
    model= "openai/gpt-oss-20b",
    api_key= os.getenv("GROQ_API_KEY"),
    temperature= 0.3,
    max_tokens= 1000,
)

In [ ]:
prompt = {
    "system": "You are a helpful assistant that summarizes news articles.",
    "user": "Summarize the following article content:\n\n{content}",
}

def summarize_article(content):
    response = groq_llm.invoke(content)
    return response

In [ ]:
summary = summarize_article(f"""
                            You are a helpful assistant that summarizes news articles.
                            Summarize the following article content:{read_website_content(["https://www.bbc.com/future/article/20241122-ai-deepfakes-is-there-something-special-about-the-human-voice"])[0].page_content}
                            """
    )

In [ ]:
read_website_content("https://www.bbc.com/future/article/20241122-ai-deepfakes-is-there-something-special-about-the-human-voice")[0].page_content

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback

In [ ]:
system_message_news_explainer = """
You are a technical news explainer.

You will receive the full content of a technical news article (technology, AI, software, hardware, cloud, cybersecurity, science, engineering).

Your responsibilities:

1) Explain the article

Explain what happened, why it matters, and how it works

Break down technical concepts, jargon, and acronyms

Add necessary background for understanding

Assume the user is technical but may not know this specific domain

2) Summarize on request

If asked to summarize:

Provide a concise summary by default

Provide a detailed/technical summary if explicitly requested

Do not add opinions unless asked

3) Answer follow-up questions

Maintain full conversational context

Answer using:

Information from the article

Logical technical inference

If something is unclear or not stated, say so explicitly

4) Technical depth control

Adjust depth based on the question:

“Explain” → conceptual

“How does it work?” → technical mechanism

“Why?” → trade-offs and reasoning

“Implications” → engineering, business, or ecosystem impact

Rules

Be precise and factual

Do not hallucinate

Clearly separate facts vs inference

Avoid marketing language

Use bullets, diagrams-in-text, or stepwise explanations when helpful

Output style

Clear, structured, and technical

Concise but complete

No emojis

No fluff

Initial response behavior

After the article is provided:

Confirm understanding

Give a high-level technical explanation

Offer next actions:

Summary

Deep dive

Architecture explanation

Pros/cons

Implications

Follow-up questions"""


prompt_template = PromptTemplate(input_variables = ["history", "input"], template = f"""
    {system_message_news_explainer}
    Conversation History:
    {{history}}
    User Question:
    {{input}}
    """)

In [ ]:
window_memory_100 = ConversationBufferWindowMemory(k=100)

conversation_chain = ConversationChain(
    llm = groq_llm,
    memory = window_memory_100,
    prompt = prompt_template,
    verbose = False
)

In [ ]:
conversation_chain.memory.clear()

In [ ]:
while True:
    input_text = input("Input: ")
    if input_text.lower() in ["exit", "quit"]:
        break
    
    response = conversation_chain.invoke({
        "input": input_text
    })
    
    display(Markdown(response['response']))